# routes.auth

> Pre-built authentication routes for FastHTML applications

In [ ]:
#| default_exp routes.auth

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from ship_kit.auth import hash_password, verify_password
from monsterui.all import *

## Overview

This module provides pre-built authentication routes that follow FastHTML patterns and the Answer.AI design philosophy:

- **Simple by default**: `login_route(rt)` gives you a working login page
- **Progressively customizable**: Override forms, handlers, or paths as needed
- **No magic**: All behavior is explicit and visible
- **MonsterUI integration**: Beautiful forms out of the box

## Quick Start

```python
from ship_kit.routes.auth import login_route, signup_route, logout_route
from fasthtml.common import *

app, rt = fast_app()

# Add all auth routes with defaults
login_route(rt)
signup_route(rt)
logout_route(rt)
```

## Component Visualization

To visualize MonsterUI components in this notebook during development, run the setup cell below. This will create a JupyUvi server that can render components with proper MonsterUI styling.

**Note**: The Show function requires a running JupyUvi server. If components aren't displaying, make sure you've run the setup cell and that the server is still running on port 8003.

In [ ]:
#| eval: false
# Setup visualization for this notebook
from fasthtml.jupyter import HTMX, JupyUvi
import uuid

# Create visualization app and server
_viz_app, _viz_rt = fast_app(hdrs=Theme.blue.headers())
_viz_server = JupyUvi(_viz_app, port=8003)

def Show(component, height='auto'):
    """Display a MonsterUI component in the notebook."""
    path = f'/show/{uuid.uuid4().hex}'
    
    @_viz_app.route(path)
    def show_component():
        return component
    
    return HTMX(path, app=_viz_app, port=8003, height=height)

In [ ]:
#| eval: false
# Remember to stop the server to avoid dangling thread
_viz_server.stop()

## Login Route

In [ ]:
#| export
def login_route(rt, # FastHTML router instance
                path='/auth/login',
                redirect_to='/',
                login_form=None,
                authenticate=None,
                session_key='auth',
                **kwargs):
    """Create a login route with customizable form and authentication.
    
    Following FastHTML patterns - pass the router instance, not the app.
    
    Args:
        rt: FastHTML router instance (from fast_app())
        path: Route path (default: '/auth/login')
        redirect_to: Where to redirect after successful login (default: '/')
        login_form: Custom form component (callable returning FT)
        authenticate: Custom authentication function(email, password) -> user_dict or None
        session_key: Session key for auth data (default: 'auth')
        **kwargs: Additional arguments passed to form component
    """
    
    # Default form using MonsterUI components
    if login_form is None:
        def login_form(error=None, **form_kwargs):
            return Card(
                H2('Login'),
                Form(
                    LabelInput('Email', 
                              name='email', 
                              id='email', 
                              type='email', 
                              required=True, 
                              placeholder='you@example.com'),
                    LabelInput('Password', 
                              name='password', 
                              id='password', 
                              type='password', 
                              required=True),
                    Alert(error, variant='destructive') if error else None,
                    Button('Sign In', type='submit', cls='w-full'),
                    method='post',
                    hx_post=path,
                    hx_target='closest .card' if error else None,
                    hx_swap='outerHTML' if error else None,
                    cls='space-y-4'
                ),
                P(
                    "Don't have an account? ",
                    A('Sign up', href='/auth/signup', cls='text-primary hover:underline'),
                    cls='text-center text-sm mt-4'
                ),
                cls='max-w-md mx-auto mt-8'
            )
    
    # Default authentication (example only - replace with real implementation)
    if authenticate is None:
        def authenticate(email, password):
            # This is just an example - you should implement real authentication
            # Check against your database, verify password hash, etc.
            if email and password:  # Placeholder logic
                return {'email': email, 'id': 1}
            return None
    
    @rt(path, methods=['GET', 'POST'])
    async def login(req, sess):
        # Redirect if already authenticated
        if sess.get(session_key):
            return RedirectResponse(redirect_to, status_code=303)
        
        if req.method == 'POST':
            # Access form data correctly in FastHTML - await the form
            form = await req.form()
            email = form.get('email', '').strip()
            password = form.get('password', '')
            
            user = authenticate(email, password)
            if user:
                sess[session_key] = user
                # Return redirect for full page or just success for HTMX
                if req.headers.get('HX-Request'):
                    return Response(headers={'HX-Redirect': redirect_to})
                return RedirectResponse(redirect_to, status_code=303)
            
            # Return form with error
            return login_form(error='Invalid email or password', **kwargs)
        
        # GET request - show form
        return login_form(**kwargs)

In [ ]:
#| eval: false
def default_login_form(error=None, **kwargs):
    return Card(
        H2('Login'),
        Form(
            LabelInput('Email', 
                      name='email', 
                      id='email', 
                      type='email', 
                      required=True, 
                      placeholder='you@example.com'),
            LabelInput('Password', 
                      name='password', 
                      id='password', 
                      type='password', 
                      required=True),
            Alert(error, variant='destructive') if error else None,
            Button('Sign In', type='submit', cls='w-full'),
            method='post',
            cls='space-y-4'
        ),
        P(
            "Don't have an account? ",
            A('Sign up', href='/auth/signup', cls='text-primary hover:underline'),
            cls='text-center text-sm mt-4'
        ),
        cls='max-w-md mx-auto mt-8'
    )

In [ ]:
# Uncomment to Visualize the default login form
#Show(default_login_form(), height='500px')

### Login Examples

#### Basic Usage

The default login form includes:
- Email and password fields with proper input types
- MonsterUI styling with Card container
- Error handling with Alert component
- HTMX integration for seamless form submission
- Link to signup page

```python
# Simple login with all defaults
login_route(rt)
```

#### Custom Form Example

In [ ]:
def branded_login_form(error=None, **kwargs):
    return Container(
        Card(
            Div(
                H1('Welcome Back!', cls='text-3xl font-bold'),
                P('Sign in to continue to your account', cls='text-muted-foreground'),
                cls='text-center mb-6'
            ),
            Form(
                LabelInput('Email Address',
                          name='email', 
                          type='email', 
                          required=True, 
                          placeholder='Enter your email'),
                Div(
                    LabelInput('Password',
                              name='password', 
                              type='password', 
                              required=True),
                    A('Forgot password?', href='/auth/forgot-password', 
                      cls='text-sm text-primary hover:underline mt-1 block'),
                    cls='space-y-1'
                ),
                Alert(error, variant='destructive', cls='mb-4') if error else None,
                Button('Sign In', type='submit', cls='w-full', size='lg'),
                DividerSplit('Or'),
                P(
                    "Don't have an account? ",
                    A('Sign up', href='/auth/signup', cls='text-primary hover:underline'),
                    cls='text-center text-sm'
                ),
                method='post',
                cls='space-y-4'
            ),
            cls='max-w-md w-full'
        ),
        cls='min-h-screen flex items-center justify-center'
    )

# Use the custom form
# login_route(rt, login_form=branded_login_form)

In [ ]:
#| eval: false
# Uncomment to Visualize the branded login form
#Show(branded_login_form(), height='700px')

#### Custom Authentication

In [ ]:
# Example with database authentication
def db_authenticate(email, password):
    """Authenticate user against database."""
    # This is pseudocode - implement your actual database logic
    # user = db.query("SELECT * FROM users WHERE email = ?", email)
    # if user and verify_password(password, user['password_hash']):
    #     return {'id': user['id'], 'email': user['email'], 'name': user['name']}
    # return None
    pass

# login_route(rt, authenticate=db_authenticate)

## Signup Route

In [ ]:
#| export
def signup_route(rt, # FastHTML router instance
                 path='/auth/signup',
                 redirect_to='/',
                 signup_form=None,
                 create_user=None,
                 session_key='auth',
                 **kwargs):
    """Create a signup route with customizable form and user creation.
    
    Following FastHTML patterns - pass the router instance, not the app.
    
    Args:
        rt: FastHTML router instance (from fast_app())
        path: Route path (default: '/auth/signup')
        redirect_to: Where to redirect after successful signup (default: '/')
        signup_form: Custom form component (callable returning FT)
        create_user: Custom user creation function(form_data) -> user_dict or error_string
        session_key: Session key for auth data (default: 'auth')
        **kwargs: Additional arguments passed to form component
    """
    
    # Default form using MonsterUI
    if signup_form is None:
        def signup_form(error=None, values=None, **form_kwargs):
            values = values or {}
            return Card(
                H2('Create Account'),
                Form(
                    LabelInput('Name',
                              name='name', 
                              id='name', 
                              required=True, 
                              value=values.get('name', ''),
                              placeholder='John Doe'),
                    LabelInput('Email',
                              name='email', 
                              id='email', 
                              type='email', 
                              required=True,
                              value=values.get('email', ''),
                              placeholder='you@example.com'),
                    LabelInput('Password',
                              name='password', 
                              id='password', 
                              type='password', 
                              required=True,
                              placeholder='At least 8 characters'),
                    LabelInput('Confirm Password',
                              name='password_confirm', 
                              id='password_confirm', 
                              type='password', 
                              required=True),
                    Alert(error, variant='destructive') if error else None,
                    Button('Create Account', type='submit', cls='w-full'),
                    method='post',
                    hx_post=path,
                    hx_target='closest .card' if error else None,
                    hx_swap='outerHTML' if error else None,
                    cls='space-y-4'
                ),
                P(
                    "Already have an account? ",
                    A('Sign in', href='/auth/login', cls='text-primary hover:underline'),
                    cls='text-center text-sm mt-4'
                ),
                cls='max-w-md mx-auto mt-8'
            )
    
    # Default user creation (example only)
    if create_user is None:
        def create_user(form_data):
            """Create user from form data. Returns user dict or error string."""
            # Validate passwords match
            if form_data.get('password') != form_data.get('password_confirm'):
                return "Passwords don't match"
            
            # Validate password length
            if len(form_data.get('password', '')) < 8:
                return "Password must be at least 8 characters"
            
            # In real implementation:
            # - Check if email already exists
            # - Hash password with hash_password()
            # - Insert into database
            # - Return user object
            
            return {
                'id': 1,  # Would be from database
                'email': form_data.get('email'),
                'name': form_data.get('name')
            }
    
    @rt(path, methods=['GET', 'POST'])
    async def signup(req, sess):
        # Redirect if already authenticated
        if sess.get(session_key):
            return RedirectResponse(redirect_to, status_code=303)
        
        if req.method == 'POST':
            # In FastHTML, form data needs to be awaited
            form = await req.form()
            form_data = {k: v.strip() if isinstance(v, str) else v 
                        for k, v in form.items()}
            
            result = create_user(form_data)
            
            if isinstance(result, str):  # Error message
                return signup_form(error=result, values=form_data, **kwargs)
            
            # Success - log user in
            sess[session_key] = result
            
            if req.headers.get('HX-Request'):
                return Response(headers={'HX-Redirect': redirect_to})
            return RedirectResponse(redirect_to, status_code=303)
        
        # GET request
        return signup_form(**kwargs)

The default signup form includes:
- Name, email, password, and password confirmation fields
- Form validation with error messages
- Preserves form values on validation errors
- HTMX integration for seamless submission
- Link to login page for existing users

In [ ]:
#| eval: false  
# Visualize the default signup form
def default_signup_form(error=None, values=None, **kwargs):
    values = values or {}
    return Card(
        H2('Create Account'),
        Form(
            LabelInput('Name',
                      name='name', 
                      id='name', 
                      required=True, 
                      value=values.get('name', ''),
                      placeholder='John Doe'),
            LabelInput('Email',
                      name='email', 
                      id='email', 
                      type='email', 
                      required=True,
                      value=values.get('email', ''),
                      placeholder='you@example.com'),
            LabelInput('Password',
                      name='password', 
                      id='password', 
                      type='password', 
                      required=True,
                      placeholder='At least 8 characters'),
            LabelInput('Confirm Password',
                      name='password_confirm', 
                      id='password_confirm', 
                      type='password', 
                      required=True),
            Alert(error, variant='destructive') if error else None,
            Button('Create Account', type='submit', cls='w-full'),
            method='post',
            cls='space-y-4'
        ),
        P(
            "Already have an account? ",
            A('Sign in', href='/auth/login', cls='text-primary hover:underline'),
            cls='text-center text-sm mt-4'
        ),
        cls='max-w-md mx-auto mt-8'
    )

In [ ]:
# Uncomment to Visualize the default signup form
#Show(default_signup_form(), height='600px')

## Logout Route

In [ ]:
#| export
def logout_route(rt, # FastHTML router instance
                 path='/auth/logout',
                 redirect_to='/auth/login',
                 session_key='auth',
                 before_logout=None):
    """Create a logout route that clears the session.
    
    Following FastHTML patterns - pass the router instance, not the app.
    
    Args:
        rt: FastHTML router instance (from fast_app())
        path: Route path (default: '/auth/logout')
        redirect_to: Where to redirect after logout (default: '/auth/login')
        session_key: Session key to clear (default: 'auth')
        before_logout: Optional callback function(session) called before logout
    """
    
    @rt(path)
    def logout(req, sess):
        # Call optional callback
        if before_logout:
            before_logout(sess)
        
        # Clear auth from session
        if session_key in sess:
            del sess[session_key]
        
        # Handle HTMX requests
        if req.headers.get('HX-Request'):
            return Response(headers={'HX-Redirect': redirect_to})
        
        return RedirectResponse(redirect_to, status_code=303)

## Complete Example

Here's a complete example showing how to use all the auth routes together with custom authentication:

In [ ]:
# Example of complete auth setup
def setup_auth(rt, db):
    """Setup authentication routes with database integration."""
    
    def authenticate_user(email, password):
        """Check credentials against database."""
        user = db.get_user_by_email(email)
        if user and verify_password(password, user['password_hash']):
            return {
                'id': user['id'],
                'email': user['email'],
                'name': user['name']
            }
        return None
    
    def create_new_user(form_data):
        """Create new user in database."""
        # Validation
        if form_data['password'] != form_data['password_confirm']:
            return "Passwords don't match"
        
        if len(form_data['password']) < 8:
            return "Password must be at least 8 characters"
        
        # Check if email exists
        if db.get_user_by_email(form_data['email']):
            return "Email already registered"
        
        # Create user
        user_id = db.create_user(
            email=form_data['email'],
            name=form_data['name'],
            password_hash=hash_password(form_data['password'])
        )
        
        return {
            'id': user_id,
            'email': form_data['email'],
            'name': form_data['name']
        }
    
    # Add all routes
    login_route(rt, authenticate=authenticate_user)
    signup_route(rt, create_user=create_new_user)
    logout_route(rt)

# Usage:
# app, rt = fast_app()
# setup_auth(rt, database)

## Interactive Demo

You can create a working demo using JupyUvi:

In [ ]:
#| eval: false
# Demo app with all auth routes
# Note: This demo uses JupyUvi to run interactively in the notebook
from fasthtml.jupyter import JupyUvi

# Create demo app
demo_app, rt = fast_app(hdrs=Theme.blue.headers())

# Simple in-memory user store for demo
demo_users = {}

def demo_authenticate(email, password):
    """Demo authentication."""
    user = demo_users.get(email)
    if user and user['password'] == password:  # Don't do this in production!
        return {'email': email, 'name': user['name']}
    return None

def demo_create_user(form_data):
    """Demo user creation."""
    if form_data['password'] != form_data['password_confirm']:
        return "Passwords don't match"
    
    if form_data['email'] in demo_users:
        return "Email already exists"
    
    demo_users[form_data['email']] = {
        'name': form_data['name'],
        'password': form_data['password']  # Don't store plain passwords in production!
    }
    
    return {'email': form_data['email'], 'name': form_data['name']}

# Add auth routes using rt pattern
login_route(rt, authenticate=demo_authenticate)
signup_route(rt, create_user=demo_create_user)
logout_route(rt)

# Add a protected home page
@rt('/')
def home(sess):
    user = sess.get('auth')
    if not user:
        return RedirectResponse('/auth/login', status_code=303)
    
    return Container(
        Card(
            H1(f"Welcome, {user.get('name', 'User')}!"),
            P(f"You are logged in as {user['email']}"),
            Button('Logout', hx_get='/auth/logout', hx_push_url='true'),
            cls='max-w-2xl mx-auto mt-8'
        )
    )

# Run the demo server
print("Starting demo server...")
print("\nDemo app routes:")
print("- http://localhost:8000/auth/login - Login page")
print("- http://localhost:8000/auth/signup - Signup page") 
print("- http://localhost:8000/auth/logout - Logout")
print("- http://localhost:8000/ - Protected home page")
print("\nTry creating an account and logging in!")

# Start the server
server = JupyUvi(demo_app)

In [ ]:
#| eval: false
# Remember to stop the server when done:
server.stop()

## Advanced Customization

### Custom Session Management

In [ ]:
# Use different session keys for different user types
# login_route(rt, session_key='admin_auth', path='/admin/login')
# login_route(rt, session_key='user_auth', path='/user/login')

### Multi-tenant Authentication

In [ ]:
def tenant_authenticate(email, password, tenant_id=None):
    """Authenticate within a specific tenant."""
    # Your multi-tenant logic here
    pass

# Create tenant-specific login
# login_route(rt, 
#     path='/tenant/{tenant_id}/login',
#     authenticate=lambda e, p: tenant_authenticate(e, p, req.path_params['tenant_id'])
# )

### OAuth Login Form

In [ ]:
def oauth_login_form(error=None, **kwargs):
    """Login form with OAuth options."""
    return Card(
        H2('Sign In'),
        Div(
            Button(
                UkIcon('github'), 'Continue with GitHub',
                onclick="window.location.href='/auth/github'",
                cls='w-full mb-2'
            ),
            Button(
                'Continue with Google', 
                onclick="window.location.href='/auth/google'",
                cls='w-full'
            ),
            cls='mb-4'
        ),
        DividerSplit('OR'),
        Form(
            # Regular email/password form
            LabelInput('Email',
                      name='email', 
                      type='email', 
                      required=True),
            LabelInput('Password',
                      name='password', 
                      type='password', 
                      required=True),
            Alert(error, variant='destructive') if error else None,
            Button('Sign In with Email', type='submit', cls='w-full'),
            method='post',
            cls='space-y-4'
        ),
        cls='max-w-md mx-auto mt-8'
    )

# Use OAuth-enabled form
# login_route(rt, login_form=oauth_login_form)

## Testing Auth Routes

Here's how to test the auth routes:

In [ ]:
#| eval: false
# Test with httpx
import httpx

def test_auth_flow(base_url='http://localhost:8000'):
    """Test the complete auth flow."""
    with httpx.Client(base_url=base_url, follow_redirects=True) as client:
        # Test signup
        response = client.post('/auth/signup', data={
            'name': 'Test User',
            'email': 'test@example.com',
            'password': 'testpass123',
            'password_confirm': 'testpass123'
        })
        assert response.status_code == 200
        
        # Test login
        response = client.post('/auth/login', data={
            'email': 'test@example.com',
            'password': 'testpass123'
        })
        assert response.status_code == 200
        
        # Test protected route
        response = client.get('/')
        assert 'Welcome' in response.text
        
        # Test logout
        response = client.get('/auth/logout')
        assert response.url.path == '/auth/login'
        
    print("All tests passed!")

# Run tests against your app
# test_auth_flow()

In [ ]:
#| eval: false
# Uncomment to Visualize the OAuth login form
# Show(oauth_login_form(), height='600px')

## Summary

The auth routes module provides:

1. **Simple defaults** - Just call `login_route(rt)` to get started
2. **Progressive customization** - Override forms, handlers, or paths as needed
3. **FastHTML patterns** - Uses standard rt() decorator, req/sess parameters, and HTMX
4. **MonsterUI integration** - Beautiful forms out of the box
5. **No magic** - All code is explicit and understandable

The routes handle both regular and HTMX requests, include CSRF protection via POST methods, and integrate seamlessly with FastHTML's session management.

### Breaking Changes in v2.0

- **Functions now take `rt` parameter instead of `app`** - This follows FastHTML best practices
- Old: `login_route(app)`
- New: `login_route(rt)`